# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
import api_keys

# Use citipy to find the city from its latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

#Import CSV file and read 
city_weather = pd.read_csv("../output_data/cities.csv")
city_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


## Generate Cities List

608


['cape town',
 'beloha',
 'longyearbyen',
 'arraial do cabo',
 'varhaug',
 'lorengau',
 'troitsko-pechorsk',
 'xichang',
 'gravelbourg',
 'rikitea',
 'mataura',
 'hovd',
 'ozernovskiy',
 'havelock',
 'barrow',
 'pemberton',
 'kualakapuas',
 'port shepstone',
 'chulman',
 'nikolskoye',
 'husavik',
 'qaanaaq',
 'ushuaia',
 'songjianghe',
 'butaritari',
 'saalfeld',
 'saskylakh',
 'wattegama',
 'khatanga',
 'busselton',
 'milkovo',
 'guadalajara',
 'gossas',
 'hilo',
 'leningradskiy',
 'hobart',
 'chokurdakh',
 'batemans bay',
 'cabo san lucas',
 'port pirie',
 'port elizabeth',
 'pisco',
 'bredasdorp',
 'hofn',
 'platanos',
 'poya',
 'half moon bay',
 'kahna nau',
 'kovdor',
 'muroto',
 'magadan',
 'namibe',
 'lavrentiya',
 'tsihombe',
 'sharjah',
 'ilulissat',
 'hasaki',
 'taganak',
 'punta arenas',
 'kununurra',
 'novopokrovka',
 'port lincoln',
 'atuona',
 'kangaatsiaq',
 'puerto ayora',
 'jahrom',
 'mehamn',
 'kimbe',
 'hermanus',
 'westport',
 'macaboboni',
 'vanavara',
 'bosaso',
 

In [2]:
# Randomly select 500 cities for weather data.

random_cities = city_weather.sample(n=500)

#Visualize the selected cities DataFrame.
random_cities

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
338,338,port shepstone,2,ZA,1558378854,47,-30.74,30.45,69.54,5.32
492,492,bolobo,95,CD,1558378889,57,-2.17,16.23,83.94,2.33
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
357,357,banda aceh,100,ID,1558378858,76,5.56,95.32,84.12,11.95
446,446,henties bay,0,NaN,1558378880,91,-22.12,14.28,59.00,5.99
...,...,...,...,...,...,...,...,...,...,...
111,111,port blair,99,IN,1558378774,71,11.67,92.75,85.20,7.18
485,485,san-pedro,92,CI,1558378888,89,4.75,-6.64,78.90,3.56
302,302,alofi,75,NU,1558378845,94,-19.06,-169.92,75.20,6.93
239,239,manzhouli,35,CN,1558378819,45,49.60,117.43,45.24,13.67


In [21]:
# OpenWeatherMap API Key
api_key = api_keys.weather_api_key

# Base URL for API Call
base_url= "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" 
query_url = url + "lat="+ str(row["Lat"]) + "&lon=" + str(row["Lng"]) + "&appid=" + api_key + "&units=" + units

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [29]:
# Use a row counter for the script to run through
city_counter = 0

# Open lists for the following columns to hold data
random_cities["Temperature"] = ""
random_cities["Humidity"] = ""
random_cities["Cloudiness"] = ""
random_cities["Wind Speed"] = ""

#Loop through each city, extract required items, and print result
print('''
------------------------------
City Weather Data Retrieval
------------------------------''')

# For/ try loop for city weather tables 
for index, row in random_cities.iterrows():
    city_counter= city_counter + 1
    
    # Base URL for API Call 
    city_weather = requests.get(query_url).json()
    
    try:
        temp = city_weather["main"]["temp"]
        humid = city_weather["main"]["humidity"]
        cloud = city_weather["clouds"]["all"]
        ws = city_weather["wind"]["speed"]
        
        random_cities.set_value(index, "Temperature", temp)
        random_cities.set_value(index,"Humidity", humid)
        random_cities.set_value(index,"Cloudiness", cloud)
        random_cities.set_value(index,"Wind Speed", ws)
         
    except (KeyError, IndexError):
        print(f'Skipping bc data is missing')
        
random_cities 



------------------------------
City Weather Data Retrieval
------------------------------


AttributeError: 'DataFrame' object has no attribute 'set_value'

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [5]:
#  Get the indices of cities that have humidity over 100%.


In [6]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression